# Extracting training for forest classification

### Background
Training data is the most important part of any supervised machine learning workflow. The quality of the training data has a greater impact on the classification than the algorithm used. Large and accurate training data sets are preferable: increasing the training sample size results in increased classification accuracy [(Maxell et al 2018)](https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1433343). A review of training data methods in the context of Earth Observation is available [here](https://www.mdpi.com/2072-4292/12/6/1034).

When creating training labels, be sure to capture the spectral variability of the class, and to use imagery from the time period you want to classify (rather than relying on basemap composites). Another common problem with training data is class imbalance. This can occur when one of your classes is relatively rare and therefore the rare class will comprise a smaller proportion of the training set. When imbalanced data is used, it is common that the final classification will under-predict less abundant classes relative to their true proportion.

### Description

This notebook will extract training data from the STAC data using labelled geometries within a geojson. The default example will use the forest/non-forest labels within the 'forest_non_forest.geojson' file. To do this, we rely on a custom dea-notebooks function called collect_training_data, contained within the dea_tools.classification script. The principal goal of this notebook is to familiarise users with this function so they can extract the appropriate data for their use-case.

1. Preview the polygons in our training data by plotting them on a basemap
2. Define a feature layer function to pass to `collect_training_data`
3. Extract training data from the datacube using `collect_training_data`
4. Export the training data to disk for use in subsequent scripts

## Start

In [1]:
# clone repo and rename it to "deanotebooks" since "-" cant be used for naming
# This repo carries the newest algorithms
#!git clone https://github.com/GeoscienceAustralia/dea-notebooks.git

In [4]:
%matplotlib inline

import os
import datacube
import numpy as np
import xarray as xr
import pandas as pd
import subprocess as sp
import geopandas as gpd
from odc.stac import configure_rio, stac_load

from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs
import planetary_computer
import pystac_client
import sys
sys.path.insert(1, 'deanotebooks/Tools/dea_tools')
from bandindices import calculate_indices
from classification import collect_training_data

import warnings
warnings.filterwarnings("ignore")

# neede packages? E.g.
# !pip install odc.io dea-tools

In [2]:
!pip install odc.io dea-tools pystac-client

  Using cached odc_io-0.2.1-py3-none-any.whl (10 kB)
  Using cached dea_tools-0.2.7-py3-none-any.whl (143 kB)
  Using cached ciso8601-2.3.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (39 kB)
  Using cached lxml-4.9.3-cp311-cp311-manylinux_2_28_x86_64.whl (7.9 MB)
  Using cached rasterstats-0.19.0-py3-none-any.whl (16 kB)
  Using cached odc_ui-0.2.0a3-py3-none-any.whl (15 kB)
  Using cached OWSLib-0.29.2-py2.py3-none-any.whl (221 kB)
  Using cached jupyter_ui_poll-0.2.2-py2.py3-none-any.whl (9.0 kB)
  Using cached simplejson-3.19.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)


## Analysis parameters

* `path`: The path to the input vector file from which we will extract training data. A default geojson is provided.
* `field`: This is the name of column in your shapefile attribute table that contains the class labels. **The class labels must be integers**

In [5]:
path = 'forest_non_forest.geojson' 
field = 'class'

## Preview input data

We can load and preview our input data shapefile using `geopandas`. The shapefile should contain a column with class labels (e.g. 'class'). These labels will be used to train our model. 

> Remember, the class labels **must** be represented by `integers`.


In [6]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

,class,geometry
0,1,POINT (552956.614 5607753.008)
1,1,POINT (553172.269 5607753.008)
2,1,POINT (553387.923 5607753.008)
3,1,POINT (553603.577 5607753.008)
4,1,POINT (553819.232 5607753.008)


In [7]:
# Plot training data in an interactive map
input_data.explore(column=field)

## Extracting training data

The function `collect_training_data` takes our geojson containing class labels and extracts training data (features) from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing any `NaN` or `inf` values.

The below variables can be set within the `collect_training_data` function:

* `zonal_stats` : An optional string giving the names of zonal statistics to calculate across each polygon (if the geometries in the vector file are polygons and not points). Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', and 'min'.

In addition to the `zonal_stats` parameter, we also need to set up a datacube query dictionary for the Open Data Cube query such as `measurements` (the bands to load from the satellite), the `resolution` (the cell size), and the `output_crs` (the output projection). These options will be added to a query dictionary that will be passed into `collect_training_data` using the parameter `collect_training_data(dc_query=query, ...)`. The query dictionary will be the only argument in the **feature layer function** which we will define and describe in a moment.

* The bounding box choosen in the cell below is much bigger than the size of the training data field.

> Note: `collect_training_data` also has a number of additional parameters for handling ODC I/O read failures, where polygons that return an excessive number of null values can be resubmitted to the multiprocessing queue. Check out the [docs](https://github.com/GeoscienceAustralia/dea-notebooks/blob/2bbefd45ca1baaa74977a1dc3075d979f3e87168/Tools/dea_tools/classification.py#L580) to learn more.

In [8]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
bbox_of_interest = [8.5, 46.8, 14, 52]


time = "2020-03"#-01/2020-03-01"

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox_of_interest,
    datetime=time,
    #query={"eo:cloud_cover": {"lt": 5}} # filtering with cloud limit 5 %
)

items = search.item_collection()
print(f"Returned {len(items)} Items")

Returned 573 Items


In [10]:
# Query options
query = {
    'items': items,
    'time': time,
    'resolution': 10,
    'output_crs' : 'EPSG:3857'
}
input_data = gpd.read_file(path).to_crs('epsg:4326')

## Defining feature layers

To create the desired feature layers, we pass instructions to `collect training data` through the `feature_func` parameter. 

* `feature_func`: A function for generating feature layers that is applied to the data within the bounds of the input geometry. The 'feature_func' must accept a 'dc_query' object, and return a single xarray.Dataset or xarray.DataArray containing 2D coordinates (i.e x, y - no time dimension). e.g.

            def feature_function(query):
                dc = datacube.Datacube(app='feature_layers')
                ds = dc.load(**query)
                ds = ds.mean('time')
                return ds

Below, we will define a feature layer function.

In [11]:
def feature_layers(query):
    ds = stac_load(groupby="solar_day",
                   chunks={},
                   bands=["coastal","green","red","blue",
                          "nir","SCL","B05","B06","B07",
                          "B11","B12"],
                  **query)
    ds=ds.where(ds.SCL.isin([4, 5, 6, 7,11]))
    ds=ds.drop('SCL')
    ds = ds.median('time')
    
    ds["red_edge1"]=ds["B05"]
    ds["red_edge2"]=ds["B06"]
    ds["red_edge3"]=ds["B07"]
    ds["swir1"]=ds["B11"]
    ds["swir2"]=ds["B12"]
    ds=ds.drop(["B05","B06","B07","B11","B12"])
    # Calculate some band indices
    da = calculate_indices(ds,
                           index=['NDVI', 'LAI', 'MNDWI'],
                           drop=False,
                           collection='ga_s2_3')
    
    return da.compute()

Check if you can run the function

Now, we can pass this function to `collect_training_data`. This will take around 2 h to run all 1785 samples using 4 CPUs.

In [ ]:
%%time
column_names, model_input = collect_training_data(
    gdf=input_data,
    dc_query=query,
    ncpus=4, # choose the number of cpus
    field=field,
    zonal_stats='median',
    feature_func=feature_layers)

Taking zonal statistic: median


  0%|          | 0/1785 [00:00<?, ?it/s]

In [85]:
print(column_names)
print('')
print(np.array_str(model_input, precision=2, suppress_small=True))

['coastal', 'green', 'red', 'blue', 'nir', 'red_edge1', 'red_edge2', 'red_edge3', 'swir1', 'swir2', 'NDVI', 'LAI', 'MNDWI']



KeyError: (0, 0)

------------------
### We will skip the process of training data extraction since this will take around 2 h. We load previously extracted data.

In [12]:
training_data = 'training_data.csv'
df = pd.read_csv(training_data)
#column_names = df.drop('class', axis=1).columns.values.tolist()
#model_input = df.drop('class', axis=1)

In [13]:
print(df)
#print(column_names)

     class  coastal   green     red    blue   nir  red_edge1  red_edge2  \
0        1    787.5  1308.0  1653.0   994.5  2290     1889.0     2028.0   
1        1    959.0  1601.0  2092.0  1163.0  2789     2381.0     2493.0   
2        1    738.0  1420.0  1816.0  1140.0  2534     2108.0     2258.0   
3        1    467.0   778.5  1206.0   535.5  1707     1416.0     1533.0   
4        1    470.0   949.0  1332.0   753.0  2252     1610.0     1897.0   
..     ...      ...     ...     ...     ...   ...        ...        ...   
424      0    151.0   343.0   277.0   189.0  2246      671.0     1702.0   
425      0    287.5   585.0  1313.0   348.0  1482     1636.5     1726.0   
426      0    289.5   975.5  2265.0   508.5  3393     3201.0     3404.0   
427      0    562.0  1392.0  2301.0   860.0  3011     2518.0     2675.0   
428      0    414.0   692.0   745.0   487.0  2038     1411.0     1851.0   

     red_edge3   swir1   swir2      NDVI       LAI     MNDWI  
0       2119.0  3878.0  3038.0  0.16

---------------

## Export training data

Once we've collected all the training data we require, we can write the data to disk. This will allow us to import the data in the next step(s) of the workflow.


In [ ]:
# Set the name and location of the output file
output_file = "results/test_training_data.txt"
# Export files to disk
np.savetxt(output_file, model_input, header=" ".join(column_names), fmt="%4f")

In [40]:
pd.DataFrame(model_input, columns=column_names).to_csv("results/training_data.csv", sep=",", index=False)

---------------
The work is based on developments of the Open Data Cube Team from Geoscience Australia
Krause, C., Dunn, B., Bishop-Taylor, R., Adams, C., Burton, C., Alger, M., Chua, S., Phillips, C., Newey, V., Kouzoubov, K., Leith, A., Ayers, D., Hicks, A., DEA Notebooks contributors 2021. Digital Earth Australia notebooks and tools repository. Geoscience Australia, Canberra. https://doi.org/10.26186/145234